In [1]:
import torch
from datasets import load_dataset
from transformers import HubertForSequenceClassification, Wav2Vec2FeatureExtractor
from torchaudio.sox_effects import apply_effects_file
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf

/home/cytech/deep-learning/voice_commands/VoiceCommands/.VC/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-02-08 16:17:37.027613: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-08 16:17:37.459494: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-02-08 16:17:37.459528: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror 

In [2]:
effects = [["channels", "1"], ["rate", "16000"], ["gain", "-3.0"]]
def map_to_array(example):
    speech, _ = apply_effects_file(example["file"], effects)
    example["speech"] = speech.squeeze(0).numpy()
    return example

In [3]:
# load a demo dataset and read audio files
# dataset = load_dataset("anton-l/superb_demo", "ks", split="test")

Found cached dataset superb_demo (/home/cytech/.cache/huggingface/datasets/anton-l___superb_demo/ks/1.9.0/77d23894ff429329a7fe80f9007cabb0deec321316f8dda1a1e9d10ffa089d08)


In [4]:
# dataset

Dataset({
    features: ['file', 'audio', 'label'],
    num_rows: 8
})

In [5]:
# dataset = dataset.map(map_to_array)

Loading cached processed dataset at /home/cytech/.cache/huggingface/datasets/anton-l___superb_demo/ks/1.9.0/77d23894ff429329a7fe80f9007cabb0deec321316f8dda1a1e9d10ffa089d08/cache-817152a508c5ceb5.arrow


In [6]:
# model = HubertForSequenceClassification.from_pretrained("superb/hubert-large-superb-ks")
# feature_extractor = Wav2Vec2FeatureExtractor.from_pretrained("superb/hubert-large-superb-ks")

In [ ]:
# dataset

Dataset({
    features: ['file', 'audio', 'label', 'speech'],
    num_rows: 8
})

In [ ]:
# # compute attention masks and normalize the waveform if needed
# inputs = feature_extractor(dataset[:4]["speech"], sampling_rate=16000, padding=True, return_tensors="pt")

In [ ]:
# logits = model(**inputs).logits
# predicted_ids = torch.argmax(logits, dim=-1)
# labels = [model.config.id2label[_id] for _id in predicted_ids.tolist()]

In [ ]:
# ### IMPLEMENT TRANSFER LEARNING HERE ###
# ### Keep the same model architecture and only change the weights of the last layer ###
# ### Retrain the last layer with our own dataset ###

# model.save_pretrained("../models/hubert_large_superb_ks")

In [3]:
import torch
from datasets import load_dataset
from transformers import HubertForSequenceClassification, Wav2Vec2FeatureExtractor
from torchaudio.sox_effects import apply_effects_file
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf

2023-02-08 17:02:06.830058: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-08 17:02:07.514990: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-02-08 17:02:07.515086: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-02-08 17:02:09.533240: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

In [4]:
from torch.utils.data import DataLoader
DATA_PATH = '../data/'
COMMANDS = ["yes", "no", "up", "down", "left", "right", "on", "off", "stop", "go", "_background_noise_"]
TARGETS = ["yes", "no", "up", "down", "left", "right", "on", "off", "stop", "go", "unknown", "_background_noise_"]
map_class_to_id = {'yes':0, 'no':1, 'up':2, 'down':3, 'left':4, 'right':5, 'on':6, 'off':7, 'stop':8, 'go':9, 'unknown':10, '_background_noise_':11}

def decode_audio(audio_binary):
    # Decode WAV-encoded audio files to `float32` tensors, 
    # normalized to the [-1.0, 1.0] range. Return `float32` audio and a sample rate.
    try :
        audio, _ = tf.audio.decode_wav(contents=audio_binary)
    except :
        return None
    # Since all the data is single channel (mono), drop the `channels`
    # axis from the array.
    return tf.squeeze(audio, axis=-1)


dataset = pd.read_csv("../dataframes_under_sampled/dataset_unsa.csv")
dataset = dataset.head(20)
dataset['waveform'] = dataset.apply(lambda row: decode_audio(tf.io.read_file(DATA_PATH + row['label'] + "/" + row['filename'])), axis=1)
dataset['label'] = dataset['label'].apply(lambda x: x if x in COMMANDS else "unknown")
# dataset['label'] = dataset['label'].apply(lambda x: map_class_to_id[x])
dataset = dataset.rename(columns={'label': 'labels'})


2023-02-08 17:02:12.662167: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-02-08 17:02:12.662534: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2023-02-08 17:02:12.662819: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (Student-laptop): /proc/driver/nvidia/version does not exist
2023-02-08 17:02:12.664853: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [5]:
dataset

,filename,labels,set,waveform
0,4e6902d0_nohash_1.wav,unknown,validation,"(tf.Tensor(3.0517578e-05, shape=(), dtype=floa..."
1,a1533da4_nohash_2.wav,unknown,testing,"(tf.Tensor(0.00018310547, shape=(), dtype=floa..."
2,113b3fbc_nohash_3.wav,on,training,"(tf.Tensor(0.0012207031, shape=(), dtype=float..."
3,cc3b5b62_nohash_3.wav,left,training,"(tf.Tensor(-6.1035156e-05, shape=(), dtype=flo..."
4,96d8bb6f_nohash_0.wav,unknown,training,"(tf.Tensor(-0.00033569336, shape=(), dtype=flo..."
5,964e8cfd_nohash_0.wav,unknown,testing,"(tf.Tensor(0.00018310547, shape=(), dtype=floa..."
6,6f2f57c1_nohash_0.wav,unknown,testing,"(tf.Tensor(-0.00021362305, shape=(), dtype=flo..."
7,789e4ee7_nohash_0.wav,unknown,training,"(tf.Tensor(-0.000579834, shape=(), dtype=float..."
8,f618568f_nohash_0.wav,unknown,training,"(tf.Tensor(0.0009765625, shape=(), dtype=float..."
9,1851e33b_nohash_0.wav,right,training,"(tf.Tensor(-0.002319336, shape=(), dtype=float..."


In [6]:
dataset = dataset.drop(columns=['filename'])

In [7]:
dataset

,labels,set,waveform
0,unknown,validation,"(tf.Tensor(3.0517578e-05, shape=(), dtype=floa..."
1,unknown,testing,"(tf.Tensor(0.00018310547, shape=(), dtype=floa..."
2,on,training,"(tf.Tensor(0.0012207031, shape=(), dtype=float..."
3,left,training,"(tf.Tensor(-6.1035156e-05, shape=(), dtype=flo..."
4,unknown,training,"(tf.Tensor(-0.00033569336, shape=(), dtype=flo..."
5,unknown,testing,"(tf.Tensor(0.00018310547, shape=(), dtype=floa..."
6,unknown,testing,"(tf.Tensor(-0.00021362305, shape=(), dtype=flo..."
7,unknown,training,"(tf.Tensor(-0.000579834, shape=(), dtype=float..."
8,unknown,training,"(tf.Tensor(0.0009765625, shape=(), dtype=float..."
9,right,training,"(tf.Tensor(-0.002319336, shape=(), dtype=float..."


In [8]:
train_dataset = dataset[dataset['set'] == 'training'].drop(columns=['set']).reset_index(drop=True)
eval_dataset = dataset[dataset['set'] == 'testing'].drop(columns=['set']).reset_index(drop=True)

In [9]:
model = HubertForSequenceClassification.from_pretrained("superb/hubert-large-superb-ks")
feature_extractor = Wav2Vec2FeatureExtractor.from_pretrained("superb/hubert-large-superb-ks")

In [10]:
from torch.utils.data import DataLoader
train_dataloader = DataLoader(train_dataset, batch_size=2, shuffle=True)
eval_dataloader = DataLoader(eval_dataset, batch_size=2)

In [16]:
train_dataloader.dataset

,labels,waveform
0,on,"(tf.Tensor(0.0012207031, shape=(), dtype=float..."
1,left,"(tf.Tensor(-6.1035156e-05, shape=(), dtype=flo..."
2,unknown,"(tf.Tensor(-0.00033569336, shape=(), dtype=flo..."
3,unknown,"(tf.Tensor(-0.000579834, shape=(), dtype=float..."
4,unknown,"(tf.Tensor(0.0009765625, shape=(), dtype=float..."
5,right,"(tf.Tensor(-0.002319336, shape=(), dtype=float..."
6,unknown,"(tf.Tensor(0.00064086914, shape=(), dtype=floa..."
7,unknown,"(tf.Tensor(-0.0010681152, shape=(), dtype=floa..."
8,unknown,"(tf.Tensor(-0.0021362305, shape=(), dtype=floa..."
9,unknown,"(tf.Tensor(0.0019226074, shape=(), dtype=float..."


In [11]:
from torch.optim import AdamW

optimizer = AdamW(model.parameters(), lr=5e-5)

In [12]:
from transformers import get_scheduler

num_epochs = 3
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    name="linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps
)

In [13]:
from tqdm.auto import tqdm

progress_bar = tqdm(range(num_training_steps))

model.train()
for epoch in range(num_epochs):
        for batch in train_dataloader:
                batch = {k: v for k,v in batch.items()}
                outputs = model(**batch)
                loss = outputs.loss
                loss.backward()

                optimizer.step()
                lr_scheduler.step()
                optimizer.zero_grad()
                progress_bar.update(1)

  0%|          | 0/21 [00:00<?, ?it/s]

KeyError: 12

In [17]:
import evaluate

metric = evaluate.load("accuracy")
model.eval()
for batch in val_dataloader:
    batch = {k: v for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)

    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    metric.add_batch(predictions=predictions, references=batch["labels"])

metric.compute()

ValueError: Evaluation module cache file doesn't exist. Please make sure that you call `add` or `add_batch` at least once before calling `compute`.

In [ ]:
# compute attention masks and normalize the waveform if needed
inputs = feature_extractor(dataset[:4]["waveform"], sampling_rate=16000, padding=True, return_tensors="pt")